<div align="center"><img src="./images/DLI_Header.png"></div>

# NVSHMEM Histogram: Duplicated Approach

In this notebook you are going to use NVSHMEM to run a histogram program. You will be copying the full histogram to every GPU, and then doing an in-place reduction for the final result.

## Objectives

By the time you complete this notebook you will:

- Be able to write a fully functional histogram program that operates over multiple GPUs using NVSHMEM.

## Histogram Introduction

Let's look at a problem that is somewhat similar to the previous one but slightly more complicated: constructing a histogram. That is, given an array of $N$ integers, and a set of $M$ integer ranges, count how many elements in the array belong in each of the $M$ ranges. Without much loss of generality we will specify that the integers are positive in the range $[0, K-1]$ and that the ranges or buckets are evenly linearly spaced (and for simplicity $K$ is evenly divisible by $M$), so that the first bucket covers $[0, K\, /\, M - 1]$, the second covers $[K\, /\, M, 2K\, /\, M - 1]$, etc.

Before we begin refactoring the code for multiple GPUs, we'll start with an example of the code using a single GPU. The simplest way to solve this problem is again using atomics. We loop through the array and, for each element in the array, calculate which bucket it should fall into; given some integer $n$ the index of the histogram array it belongs to is $(nM)\, /\, K$. We then atomically increment the counter for that bucket. Inspect [the code](code/histogram.cpp) and then run it to see what kind of output we get. Feel free to adjust the parameters if you like (though be careful about 32-bit integer overflow if you make the numbers too large).

In [ ]:
!nvcc -x cu -arch=sm_70 -o histogram code/histogram.cpp
!./histogram

## NVSHMEM Implementation for the Duplicated Approach

One way to distribute this across multiple GPUs would be the same approach we used for the $\pi$ estimator: given $N$ integers we can distribute them evenly across our GPUs. Then we can do a reduction across all PEs. We will call this the "duplicated" approach since a copy of the full histogram exists on all GPUs. We will name the first step that increments the histogram buckets the "tabulation" step, and the second step that combines the results across all PEs the "combination" step and time them separately (for comparison against the next method).

<center><img src="images/histogram_duplicated_approach.png" width="1000"></center>

We'll use the reduction API [nvshmem_int_sum_reduce()](https://docs.nvidia.com/nvshmem/api/gen/api/collectives.html#sum) to reduce over all buckets of the histogram:

```cpp
nvshmem_int_sum_reduce(team, destination, source, nelems);
```

If `destination == source`, then this becomes an in-place reduction and this is legal to do in NVSHMEM; this has the benefit of being cleaner than creating a temporary destination buffer, so we recommend doing so in this exercise.

## Exercise: NVSHMEM Multi GPU Histogram

Look for the FIXMEs in [exercises/histogram_step1.cpp](exercises/histogram_step1.cpp) to guide your work.

Feel free to consult [the solution](solutions/histogram_step1.cpp) if you need help.

### Run the Code

In [ ]:
!nvcc -x cu -arch=sm_70 -rdc=true -I $NVSHMEM_HOME/include -L $NVSHMEM_HOME/lib -lnvshmem -lcuda -o histogram_step1 exercises/histogram_step1.cpp
!nvshmrun -np $NUM_DEVICES ./histogram_step1

## Next

In the next notebook we will continue with our histogram program. This time, instead of duplicating the histogram across GPUs and then reducing it, we will distribute parts of the histogram to each GPU and concatenate them. In addition to increasing our NVSHMEM capabilities, the refactor will also give us a chance to observe performance trade-offs we should consider when distributing work to multiple GPUs.

Please open the next notebook: [_NVSHMEM Histogram: Distributed Approach_](11_Histogram-Dist.ipynb).